### Notebook explaining and going over the tiling process.
Note: Not modular intentionally for now.

In [1]:
#Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#For GeoTIFF images
import rasterio
from rasterio.plot import show
from osgeo import gdal

In [3]:
#For Visualisation
from matplotlib import pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure
from PIL import Image

In [4]:
# Model Building
# import ultralytics
# from ultralytics import YOLO
# import labelme2yolo

In [5]:
# Others
import os
import shutil
import zipfile
from tqdm import tqdm

In [6]:
pre_event_image = './Pre_Event_San_Juan.tif'
post_event_image ='./Post_Event_San_Juan.tif'

# Tile Generation from the Pre Event Image

In [7]:
pre_event_file = gdal.Open(pre_event_image)

In [8]:
type(pre_event_file)

osgeo.gdal.Dataset

In [9]:
width = pre_event_file.RasterXSize #number of pixel columns

In [10]:
height = pre_event_file.RasterYSize #number of pixel rows

In [11]:
num_bands = pre_event_file.RasterCount #number of spectral bands, 3 in this case

In [12]:
#specify the grid size of each tile (likely in terms of pixel count along height and width
grid_x = 512
grid_y = 512

In [13]:
num_tiles_x = width // grid_x
num_tiles_y = height // grid_y
print(f"Total number of tiles: {num_tiles_x * num_tiles_y}")

Total number of tiles: 10730


In [22]:
output_dir = "Pre_Event_Grids_In_TIFF"
os.makedirs(output_dir, exist_ok=True)

In [29]:
#tells what is the format in which pixel values are stored in each band
#will return an integer where the mapping can be seen here: http://ikcest-drr.osgeo.cn/tutorial/k8023
#in this case, it should be '1' which corresponds to an 8-bit unsigned integer
valuetype = pre_event_file.GetRasterBand(1).DataType 

In [30]:
#options for handling how each tile is stored 
#these are passed to a gdal driver, as below
#most of these are meant for handling the compression of the tile file during output and how its eventually stored
options = ['COMPRESS=DEFLATE', 'PREDICTOR=2', 'TILED=YES']

In [33]:
#generates a 6-element tuple which contains the information on how each pixel in the image is mapped to actual spatial positions
#the tuple information is as follows:
#(ulx, xres, xskew, uly, yskew, yres)
#ulx: X-coordinate of the upper left corner of the image in the real-world coordinate system
#xres: Pixel width in real-world units (here, in meters). Positive values indicate increasing X to the right.
#xskew: Skew in the X direction (usually 0).
#uly: Y-coordinate of the upper left corner of the image in the real-world coordinate system.
#yskew: Skew in the Y direction (usually 0).
#yres: Pixel height in real-world units (here, in meters). Positive values indicate increasing Y downwards.

#the same information can also be obtained if the GeoTIFF is read as a rasterio.io.DatasetReader object.
#say, this object variable is called src, then src.transform will give us this same information but in a matrix form
#then, the product of the src.transform matrix with any pixel location index will give the spatial location of that pixel in metres
#the src.transform matrix is simply [[xres,xskew,ulx],[yskew,yres,uly]]
geotransform = list(pre_event_file.GetGeoTransform())

In [35]:
#specifies the coordinate reference system (CRS) which defines the origin for the spatial coordiantes obtained after applying the transformation matrix above
projection = pre_event_file.GetProjection()

In [36]:
# Iterate over each tile and save as a separate TIFF image
for i in range(num_tiles_x):
    for j in range(num_tiles_y):
        x_offset = i * grid_x
        y_offset = j * grid_y
        
        tile_width = min(grid_x, width - x_offset)
        tile_height = min(grid_y, height - y_offset)
        
        tile = [] #will collect the pixel values for the three bands for the current tile
        for band in range(1, num_bands + 1):
            # read pixel values for a specific tile and a specific band
            tile_data = pre_event_file.GetRasterBand(band).ReadAsArray(x_offset, y_offset, tile_width, tile_height)
            tile.append(tile_data)
        
        output_file = os.path.join(output_dir, f"tile_{i}_{j}.tif")
        
        # Create an output TIFF file with same CRS and band values range
        driver = gdal.GetDriverByName("GTiff") #driver for handling GeoTIFF files
        out_ds = driver.Create(output_file, tile_width, tile_height, num_bands, 
                       valuetype, options=options)

        # Set the geotransform
        tilegeotransform = geotransform
        tilegeotransform[0] = geotransform[0] + x_offset * geotransform[1]
        tilegeotransform[3] = geotransform[3] + y_offset * geotransform[5]
        out_ds.SetGeoTransform(tuple(tilegeotransform))
        
        # Set the projection
        out_ds.SetProjection(projection)
        
        # Write each band to the output file
        for band in range(1, num_bands + 1):
                out_band = out_ds.GetRasterBand(band)
                out_band.WriteArray(tile[band - 1])
        
        
        # Close the output file
        out_ds = None
                
print("Tiles generation completed.")

Tiles generation completed.


# Tile Generation from the Post Event Image

In [22]:
!wget https://challenge.ey.com/api/v1/storage/admin-files/Post_Event_San_Juan.tif -O Post_Event_San_Juan.tif

--2024-02-15 16:05:41--  https://challenge.ey.com/api/v1/storage/admin-files/Post_Event_San_Juan.tif
Resolving challenge.ey.com (challenge.ey.com)... 52.236.158.32
Connecting to challenge.ey.com (challenge.ey.com)|52.236.158.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1310494499 (1.2G) [application/octet-stream]
Saving to: ‘Post_Event_San_Juan.tif’

Post_Event_San_Juan 100%[===================>]   1.22G  64.7MB/s    in 23s     

2024-02-15 16:06:15 (55.2 MB/s) - ‘Post_Event_San_Juan.tif’ saved [1310494499/1310494499]



In [23]:
!rm -rf ./Post_Event_Grids_In_TIFF

In [24]:
post_event_file = gdal.Open(post_event_image)

In [26]:
width = post_event_file.RasterXSize 
height = post_event_file.RasterYSize 
num_bands = post_event_file.RasterCount
print(width, height, num_bands)

38259 74602 3


In [28]:
grid_x = 512
grid_y = 512

In [29]:
num_tiles_x = width // grid_x
num_tiles_y = height // grid_y
print(f"Total number of tiles: {num_tiles_x * num_tiles_y}")

Total number of tiles: 10730


In [30]:
output_dir = "Post_Event_Grids_In_TIFF"
os.makedirs(output_dir, exist_ok=True)

In [31]:
valuetype = post_event_file.GetRasterBand(1).DataType 

options = ['COMPRESS=DEFLATE', 'PREDICTOR=2', 'TILED=YES']

geotransform = list(post_event_file.GetGeoTransform())

projection = post_event_file.GetProjection()

In [32]:
# Iterate over each tile and save as a separate TIFF image
for i in tqdm(range(num_tiles_x)):
    for j in range(num_tiles_y):
        x_offset = i * grid_x
        y_offset = j * grid_y
        
        tile_width = min(grid_x, width - x_offset)
        tile_height = min(grid_y, height - y_offset)
        
        tile = [] #will collect the pixel values for the three bands for the current tile
        for band in range(1, num_bands + 1):
            # read pixel values for a specific tile and a specific band
            tile_data = post_event_file.GetRasterBand(band).ReadAsArray(x_offset, y_offset, tile_width, tile_height)
            tile.append(tile_data)
        
        output_file = os.path.join(output_dir, f"tile_{i}_{j}.tif")
        
        # Create an output TIFF file with same CRS and band values range
        driver = gdal.GetDriverByName("GTiff") #driver for handling GeoTIFF files
        out_ds = driver.Create(output_file, tile_width, tile_height, num_bands, 
                       valuetype, options=options)

        # Set the geotransform
        tilegeotransform = geotransform
        tilegeotransform[0] = geotransform[0] + x_offset * geotransform[1]
        tilegeotransform[3] = geotransform[3] + y_offset * geotransform[5]
        out_ds.SetGeoTransform(tuple(tilegeotransform))
        
        # Set the projection
        out_ds.SetProjection(projection)
        
        # Write each band to the output file
        for band in range(1, num_bands + 1):
                out_band = out_ds.GetRasterBand(band)
                out_band.WriteArray(tile[band - 1])
        
        
        # Close the output file
        out_ds = None
                
print("Tiles generation completed.")

100%|██████████| 74/74 [08:05<00:00,  6.56s/it]

Tiles generation completed.
